#### The following code is example of how to train a linear regression model using tensorflow core:

In [10]:
import tensorflow as tf
print(tf.__version__)
# Make the weights and baises
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)

1.3.0


In [13]:
#Make x be inputs and y be the expected outputs in order to train
x = tf.placeholder(tf.float32)
linear_model = W * x + b
y = tf.placeholder(tf.float32)
#Set your input data andexpected output data
x_train = [1, 2, 3, 4]
y_train = [0, -1, -2, -3]

In [14]:
# Make loss function and put it into gradient descent to be reduced
loss = tf.reduce_sum(tf.square(linear_model - y)) # sum of the squares
# initalize optimizer with learning rate
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

In [15]:
# initalize variables and make a session
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init) 

In [16]:
#run training session with 1000 epoch's 
for i in range(1000):
  sess.run(train, {x: x_train, y: y_train})

# evaluate training accuracy
curr_W, curr_b, curr_loss = sess.run([W, b, loss], {x: x_train, y: y_train})
print("W: %s b: %s loss: %s"%(curr_W, curr_b, curr_loss))

W: [-0.9999969] b: [ 0.99999082] loss: 5.69997e-11


#### The following code is example of how to train a linear regression model using tensorflow estimator (higher level of abstratction):

In [17]:
import tensorflow as tf
# NumPy is often used to load, manipulate and preprocess data.
import numpy as np

# Declare list of features. We only have one numeric feature. There are many
# other types of columns that are more complicated and useful.
feature_columns = [tf.feature_column.numeric_column("x", shape=[1])]

# An estimator is the front end to invoke training (fitting) and evaluation
# (inference). There are many predefined types like linear regression,
# linear classification, and many neural network classifiers and regressors.
# The following code provides an estimator that does linear regression.
estimator = tf.estimator.LinearRegressor(feature_columns=feature_columns)

# TensorFlow provides many helper methods to read and set up data sets.
# Here we use two data sets: one for training and one for evaluation
# We have to tell the function how many batches
# of data (num_epochs) we want and how big each batch should be.
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7, 0.])
input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=None, shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_eval}, y_eval, batch_size=4, num_epochs=1000, shuffle=False)

# We can invoke 1000 training steps by invoking the  method and passing the
# training data set.
estimator.train(input_fn=input_fn, steps=1000)

# Here we evaluate how well our model did.
train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics: %r"% train_metrics)
print("eval metrics: %r"% eval_metrics)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_summary_steps': 100, '_session_config': None, '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_tf_random_seed': 1, '_log_step_count_steps': 100, '_model_dir': 'C:\\Users\\trest\\AppData\\Local\\Temp\\tmp66szg7us'}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\trest\AppData\Local\Temp\tmp66szg7us\model.ckpt.
INFO:tensorflow:loss = 10.0, step = 1
INFO:tensorflow:global_step/sec: 683.5
INFO:tensorflow:loss = 0.0616257, step = 101 (0.146 sec)
INFO:tensorflow:global_step/sec: 976.375
INFO:tensorflow:loss = 0.0128946, step = 201 (0.102 sec)
INFO:tensorflow:global_step/sec: 846.233
INFO:tensorflow:loss = 0.00500665, step = 301 (0.118 sec)
INFO:tensorflow:global_step/sec: 1161.35
INFO:tensorflow:loss = 0.00163519, step = 401 (0.102 sec)
INFO:tensorflow:global_step/sec: 871.558
INFO:t

### Simple Mnist Reader

In [40]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

# Import data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

# Create the model
x = tf.placeholder(tf.float32, [None, 784])
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
y = tf.matmul(x, W) + b

# make placeholder for correct outputs
y_ = tf.placeholder(tf.float32, [None, 10])

#implement cost function as a cross entropy formula (slightly modified for tensorflow)
# normally: tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(tf.nn.softmax(y)), reduction_indices=[1]))
#softmax is a function that normalizes results and makes outputs into probability of the answer (so outputs sum to 1)
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

#initalize variables and session
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

# Train the model with batch sides of 100
for _ in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})


# Test trained model
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print("Acurracy:", sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Acurracy: 0.9156


In [41]:
sess = tf.InteractiveSession()

NameError: name 'tempfile' is not defined

## Cheat Sheet

In [10]:
#make a session and run it
sess=tf.session
sess.run()

#make const
c_float= tf.constant(0.0)
c_integer=tf.constant(0)
#make variable
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)

#initalize variables
init = tf.global_variables_initializer()
sess.run(init)

#make placeholder (takes in a map to initalize placeholders when session is ran)
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W * x + b
sess.run(linear_model, {x: [1, 2, 3, 4]})

#more placeholders this one takes an unknown amount of inputs (the None) of dimension: #num_inputs x 784
tf.placeholder(tf.float32, [None, 784])


#tensor flow equivalent to a lambda variable/ higher order function
adder_node = a + b 
add_and_triple = adder_node * 3.

#pass in variables to run session and some function (adder_node in this case)
sess.run(adder_node, {a: 3, b: 4.5})
sess.run(adder_node, {a: [1, 3], b: [2, 4]})

#make optimization model and pass it some cost function
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

#get the biggest of all the results (usually the choice of the NN output layer)
tf.argmax(y, 1)

#multiply matrix's
tf.matmul(x,W)

#check if predictitons are correct (True or False result)
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

array([ 3.,  7.], dtype=float32)